In [2]:
import json
from openai import OpenAI

In [3]:
with open('../data/recipes.json') as file:
    recipes_data = json.load(file)

In [15]:
import random
recipes_with_descriptions = [recipe for recipe in recipes_data.values() if recipe.get('description') is not None]
recipes_without_descriptions = [recipe for recipe in recipes_data.values() if recipe.get('description') is None]
client = OpenAI()
def recipe_to_str(recipe):
        return f"""
    Title: {recipe.get('title')}
    category: {recipe.get('category')}
    instructions: {recipe.get('instructions')}
    ingredients: {recipe.get('ingredients')}
    """
    
def get_recipe_description(recipe, examples = []):
        messages = [
            {"role": "system", "content": "You are a foodwriter, adept at writing concise and helpful descriptions for recipes. Use the information given but also feel free to bring in your own cooking knowledge"},
            {"role": "system", "content": "Rules: Keep your responses to less than 200 characters. Do not repeat the name of the dish. Do not sound too similar to the other examples."},
        ]
        for example in examples:
            messages += [
                {"role": "user", "content": "Write a brief description of the following dish:" + recipe_to_str(example)},
                {"role": "system", "content": example.get('description')}
            ]
        
        messages += [
            {"role": "user", "content": "Write a brief description of the following dish:" + recipe_to_str(recipe)},
        ]
        
        completion = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages=messages
        )
        
        return completion.choices[0].message.content
    
examples = recipes_with_descriptions
for recipe in recipes_without_descriptions[:10]:
    description = get_recipe_description(recipe, examples)
    recipes_data[recipe['title']]['description'] = description
    recipe['description'] = description
    examples.append(recipe)
    examples = random.sample(recipes_with_descriptions, 5)
    

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 4352 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
examples

In [14]:
get_recipe_description(recipes_without_descriptions[0])

ChatCompletionMessage(content='This grain bowl features roasted mushrooms, halloumi cheese, and a medley of raw vegetables on a bed of barley, topped with a tangy yogurt and olive sauce. A flavorful and satisfying meal or side dish.', role='assistant', function_call=None, tool_calls=None)

In [7]:
get_recipe_description(recipes_without_descriptions[0])

ChatCompletionMessage(content="This grain bowl features roasted mushrooms, halloumi, and a vibrant mix of raw vegetables. Topped with lemony barley and a yogurt-olive dressing, it's a delicious and filling meal.", role='assistant', function_call=None, tool_calls=None)

In [8]:
recipes_without_descriptions[0]

{'author': 'Ali Slagle',
 'canonical_url': 'https://cooking.nytimes.com/recipes/1023492-roasted-mushroom-and-halloumi-grain-bowl',
 'category': 'dinner, lunch, weeknight, main course, side dish',
 'host': 'cooking.nytimes.com',
 'image': 'https://static01.nyt.com/images/2022/09/20/dining/as-roasted-mushroom-grain-bowl/merlin_213230274_1d9c89ae-4388-4b4c-8db8-e47f24defbc3-mediumSquareAt3X.jpg',
 'ingredients': ['1 pound cremini or button mushrooms, sliced 1/4-inch thick',
  '1 bunch scallions, trimmed and cut into 1-inch lengths',
  '8 to 9 ounces halloumi, torn into bite-size pieces',
  '4 tablespoons extra-virgin olive oil, plus more for serving',
  'Kosher salt (such as Diamond Crystal) and black pepper',
  '1 cup pearled barley or farro',
  '1 cup Greek yogurt or sour cream',
  '1/2 cup Kalamata olives, pitted and coarsely chopped',
  '1 lemon',
  '1 1/2 to 2 cups thinly sliced raw vegetables (such as brussels sprouts, celery, fennel, cucumbers and radishes)',
  '1/3 cup torn mint, 